<a href="https://colab.research.google.com/github/Etiene-vc/Alura-AI-Immersion/blob/main/Find_na_aula_da_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

****************************************************
** Find nas Aulas da Alura: Encontre respostas na Imersão IA! **
****************************************************

Tem dúvidas sobre a Imersão IA da Alura? 🤔  Use o Find nas Aulas da Alura para encontrar respostas rápidas nas transcrições das aulas!

Como usar:

1. Digite sua pergunta.
2. Veja a resposta e a indicação da aula e momento exato do conteúdo.
3. Se a resposta não te ajudar, o programa te mostrará outras opções.

Exemplo:

Você: Como gerar o API Key?

Find nas Aulas da Alura: Veja a aula 4, no instante 1:10:49: "...para conseguirmos essa API KEY, vamos no lado oposto do GetCode. No canto superior esquerdo, você clica lá..."

Você: Não era isso.

Find nas Aulas da Alura: Veja a aula 2, no instante 0:45:12: "...a API KEY é gerada automaticamente após a criação da sua conta..."

Você:  Legal, era isso!

Você: sair

😉 Pronto! Agora é só usar o Find nas Aulas da Alura.

In [ ]:
!pip install -U -q google-generativeai nltk

# Importações e configurações iniciais
import pandas as pd
import google.generativeai as genai
import nltk
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

GOOGLE_API_KEY = "COLE SUA CHAVE APIKEY"
genai.configure(api_key=GOOGLE_API_KEY)

# Carregar o arquivo CSV
df = pd.read_csv('trans.csv', names=['aula', 'horário', 'transcrição'], header=0)

def resumir_texto(texto):
    """
    Resume o texto extraindo apenas a frase mais importante.
    """
    frases = sent_tokenize(texto)
    palavras = [palavra.lower() for frase in frases for palavra in nltk.word_tokenize(frase) if palavra.isalnum()]
    frequencia_palavras = FreqDist(palavras)

    stop_words = set(stopwords.words('portuguese'))
    pontuacao = {',', '.', '!', '?', ';', ':'}

    pontuacao_palavras = [(palavra, frequencia) for palavra, frequencia in frequencia_palavras.items() if palavra not in stop_words and palavra not in pontuacao]

    frases_pontuadas = []
    for frase in frases:
        pontuacao = sum([frequencia for palavra, frequencia in pontuacao_palavras if palavra in frase])
        frases_pontuadas.append((frase, pontuacao))

    frases_pontuadas.sort(key=lambda x: x[1], reverse=True)
    return frases_pontuadas[0][0]  # Retorna apenas a frase mais importante

# Resumir as transcrições para o mínimo
transcricoes_minimas = []
for i in range(len(df)):
  resumo = resumir_texto(df['transcrição'][i])
  transcricoes_minimas.append(resumo)

# Combinar as transcrições resumidas em um único texto
texto_transcricoes = ' '.join(transcricoes_minimas)

def buscar_resposta(pergunta):
    response = genai.generate_text(
        model="models/chat-bison-001",
        prompt=f"""
        Responda à seguinte pergunta com base neste texto:
        ```
        {texto_transcricoes}
        ```

        Pergunta: {pergunta}
        """,
        temperature=0.0,
        max_output_tokens=100
    )
    return response.text.strip()

# Loop principal
while True:
    pergunta = input("Faça sua pergunta (ou digite 'sair' para encerrar): ")
    if pergunta.lower() == 'sair':
        break

    resposta = buscar_resposta(pergunta)
    print(f"Resposta:\n{resposta}\n")